# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA (Low-Rank Adaptation)
* Model: distilbert-base-uncased
* Evaluation approach:  Use the Hugging Face Trainer class and evaluate library to measure accuracy
* Fine-tuning dataset: IMDb movie reviews dataset for binary sentiment classification (positive/negative)

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from datasets import load_dataset

# Load IMDb dataset
dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
from transformers import Trainer, TrainingArguments
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    accuracy = (predictions == labels).astype(np.float32).mean().item()
    return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir="/tmp/results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    logging_dir="/tmp/logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset["test"].select(range(1000)),  # small subset
    compute_metrics=compute_metrics,
)

# Evaluate the base model
base_results = trainer.evaluate()
print("Base Model Performance:", base_results)

Base Model Performance: {'eval_loss': 0.5534008145332336, 'eval_accuracy': 1.0, 'eval_runtime': 7.6919, 'eval_samples_per_second': 130.006, 'eval_steps_per_second': 16.251}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [5]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/tmp/peft-imdb",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_dir="/tmp/logs",
    logging_steps=50
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(2000)),  # Small subset for quick run
    eval_dataset=tokenized_dataset["test"].select(range(1000)),
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.000300,0.000211,1.000000


TrainOutput(global_step=250, training_loss=0.03374852456897497, metrics={'train_runtime': 38.4669, 'train_samples_per_second': 51.993, 'train_steps_per_second': 6.499, 'total_flos': 134739406848000.0, 'train_loss': 0.03374852456897497, 'epoch': 1.0})

In [7]:
peft_model.save_pretrained("/tmp/peft-imdb")

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [8]:
from peft import AutoPeftModelForSequenceClassification

peft_loaded_model = AutoPeftModelForSequenceClassification.from_pretrained("/tmp/peft-imdb")
peft_loaded_model.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,184,260 || all params: 67,694,596 || trainable%: 1.749415861791981


In [9]:
from transformers import Trainer, TrainingArguments

eval_args = TrainingArguments(
    output_dir="/tmp/eval_peft",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True
)

peft_trainer = Trainer(
    model=peft_loaded_model,
    args=eval_args,
    eval_dataset=tokenized_dataset["test"].select(range(1000)),
    compute_metrics=compute_metrics
)

In [10]:
peft_results = peft_trainer.evaluate()
print("PEFT Model Performance:", peft_results)

PEFT Model Performance: {'eval_loss': 0.00021111691603437066, 'eval_accuracy': 1.0, 'eval_runtime': 7.5456, 'eval_samples_per_second': 132.528, 'eval_steps_per_second': 16.566}


In [11]:
print("🔹 Base model accuracy:", base_results["eval_accuracy"])
print("🔹 PEFT model accuracy:", peft_results["eval_accuracy"])

🔹 Base model accuracy: 1.0
🔹 PEFT model accuracy: 1.0
